*Relembrando...*

Arredondamento no R de números do tipo `x.5` depende da paridade da unidade de x.

### Sistemas Lineares (cap 8.1 + 8.2)

---

Lidamos com problemas do tipo, $Ax = B$, onde $A$, uma matriz, e $b$, um vetor, são dados.

1. [Eliminação Gaussiana](#Eliminação-Gaussiana)

2. [Método de Jacobi](#Método-de-Jacobi)
    * [Exercício 1](#Exercício-1:)
    * [Exercício 2](#Exercício-2:)
    * [Exercício 3](#Exercício-3:)
    * [Exercício 4](#Exercício-4:)
    * [Exercício 5](#Exercício-5:)

3. [Método da Secante](#Método-da-Secante)
4. [Gradiente Conjugado](#Gradiente-Conjugado)

## 07-08

##### [Eliminação Gaussiana](https://en.wikipedia.org/wiki/Gaussian_elimination#Computational-efficiency), $O(n^3)$

#### [Método de Jacobi](https://en.wikipedia.org/wiki/Jacobi_method)

Decompomos a matriz $A$ na forma $L + D + U$, onde $L$ é uma matriz triangular inferior, $D$ é diagonal e $U$ é triangular superior.

Logo, ficamos com: 

$\begin{eqnarray}
(L + D + U)x & = & b \\
Dx & = & b - (L + U)x \\
x_{k+1} & = & D^{-1}(b-(L+U)x_{k})
\end{eqnarray}$ 

Solução é um ponto fixo da iteração de $x_{k}$

#### Exercício 1:

Gere uma matriz quadrada $n\times n$ esparsa com $\approx 2\%$ das entradas diferentes de zero e menores que 1

A diagonal deve conter números aleatórios em $[\frac{4n}{100}, \frac{8n}{100}]$, para $n=10000$.

In [56]:
n = 10000

In [57]:
import numpy as np

def esparse_matrix(n):
    
    # gerando a matriz com aprox. 5% de entradas diferentes de 1
    M = np.random.rand(n,n)
    M = M * (M > 0.95).astype(int)
    
    # preenchendo a diagonal
    np.fill_diagonal(M, 0)
    d = np.diag(np.random.uniform(n*4/100,n*8/100, n))
    
    return M + d

In [58]:
%%time
A = esparse_matrix(n)

CPU times: user 2.62 s, sys: 2.19 s, total: 4.81 s
Wall time: 4.81 s


In [59]:
A

array([[673.76175011,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.98175907, 482.15734102,   0.        , ...,   0.95485156,
          0.97431919,   0.96082707],
       [  0.        ,   0.        , 477.0665167 , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ..., 695.12997308,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
        461.59457421,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        , 557.8317368 ]])

**Exercício 2**: 

Gere um vetor aleatório $x$ e calcule $b=Ax$

In [60]:
x = 2*np.ones(n)
x.shape

(10000,)

In [61]:
x

array([2., 2., 2., ..., 2., 2., 2.])

In [62]:
b = np.inner(A,x)
b.shape

(10000,)

In [63]:
b

array([2346.81092724, 1930.73494124, 1953.34411113, ..., 2292.3611657 ,
       1936.64177518, 2110.28308954])

**Exercício 3**: 

Desenvolva $Ax=b$ por escalonamento e por Jacobi e compare os tempos

In [64]:
def Jacobi(A, b, x, threshold):
    
    """
    Resolove o sistema linear pelo método de Jacobi.
    
    :param A: matriz nxn
    :param b: vetor nx1
    :param x: vetor inicial
    :param threshold: número máximo de iterações
    
    :returns: vetor solução do sistema, número de iterações
    """
    
    U = np.triu(A)
    np.fill_diagonal(U, 0)
    
    L = np.tril(A)
    np.fill_diagonal(L, 0)
    
    D = np.diag(np.diag(A))
    D_inv = np.linalg.inv(D) 
    
    t = 0
    while t < threshold:
        
        x = np.inner(D_inv, b - np.inner((L+U), x))
        t += 1
    
    return x, t

In [65]:
x0 = np.ones(n)
t_max = 200

In [66]:
%%time

x, t = Jacobi(A, b, x0, t_max)

CPU times: user 31min 56s, sys: 12min 49s, total: 44min 46s
Wall time: 4min 39s


In [67]:
x

array([2., 2., 2., ..., 2., 2., 2.])

In [68]:
np.inner(A,x)

array([2346.81092724, 1930.73494124, 1953.34411113, ..., 2292.3611657 ,
       1936.64177518, 2110.28308954])

In [90]:
def elim_Gauss(A, b):
    
    """
    Resolove o sistema linear pelo método de eliminação gaussiana.
    
    :param A: matriz nxn
    :param b: vetor nx1
    
    :returns x: vetor solução do sistema
    """

    A = np.column_stack([A, b])        
    m = A.shape[1] # Número de colunas da matriz A|b
    n = A.shape[0]
    
    for k in range(0, n):
        
        # Buscamos o pivô da coluna k
        elem = abs(A[k][k])
        max_row = k
        
        for i in range(k+1, n):
            
            if abs(A[i][k]) > elem:
                
                elem = abs(A[i][k])
                max_row = i

        # Trocamos as linhas i e k
        A[[k,i]] = A[[i,k]]
        
        # Zeramos os elementos abaixo do pivô da coluna k
        for i in range(k+1, n):
            
            if A[i][k] == 0:

                pass
            
            else:
                
                c = -A[i][k]/A[k][k] # coeficiente

                for j in range(k, m):

                    A[i][j] += c * A[k][j]
                
         
        # Cálculo do vetor x
        x = np.zeros(n)
        
        for i in range(n-1, -1, -1):
            
            x[i] = A[i][n]/A[i][i]
            
            for k in range(i-1, -1, -1):
                
                A[k][n] -= A[k][i] * x[i]
        
        return x

In [91]:
%%time

y = elim_Gauss(A, b)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in double_scalars


CPU times: user 1min 18s, sys: 312 ms, total: 1min 19s
Wall time: 1min 19s


In [92]:
y

array([nan, nan, nan, ..., nan, nan, nan])

#### Exercício 4:
Exiba uma matriz onde o método de Jacobi não converge

In [35]:
A1 = np.array([(1,2,3), (4,5,6), (7,8,9)])
A1

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [37]:
b1 = np.array([1,1,1])
b1

array([1, 1, 1])

In [40]:
x1 = b1

In [44]:
%%time

y1, t1 = Jacobi(A1, b1, x1, t_max)

CPU times: user 4 ms, sys: 8 ms, total: 12 ms
Wall time: 4.21 ms


In [45]:
y1

array([7.20368770e+81, 3.91009795e+81, 3.54414353e+81])

In [49]:
np.inner(A1, y1)

array([2.56563142e+82, 6.96301017e+82, 1.13603889e+83])

#### Exercício 5:
    
Mostre que se a matriz A é diagonal dominante então o método de Jacobi converge.

**R:** O método converge se $e_k = (x_k - x_{k-1}) \rightarrow 0$ quando $k \rightarrow \infty$.

Seja a matriz quadrada A $n \times n$, decomposta pela fatoração LU nas matrizes $D$, $L$ e $U$, e seja $x_{0}$ nosso vetor inicial.

A cada iteração, temos $x_{k} = D^{-1}(L+U)x_{k-1} + D^{-1}b$. Para simplificar a notação, denotaremos $M = D^{-1}(L+U)$. 

Ou seja, $x_{k} = Mx_{k-1} + D^{-1}b$ e M é uma matriz quadrada $n \times n$. 

Logo, o erro pode ser escrito como:

$e_k = M(x_{k-1} - x_{k-2}) = Me_{k-1}$

Para simplificar a notação, denotaremos $M = D^{-1}(L+U)$. 
Fazendo a transformação acima sucessivamente, temos:

$e_{k-1} = Me_{k-2} \rightarrow e_{k} = Me_{k-1} = M^{2}e_{k-2}$

$e_{k-2} = Me_{k-3} \rightarrow e_{k} = M_{2}e_{k-2} = M^{3}e_{k-2}$

...

$e_{1} = Me_{0} \rightarrow e_{k} = M_{k-1}e_{1} = M^{k}e_{0}$

Usando a norma máximo (norma $p$< com $p=\inf$) e desigualdade triangular para o produto $M_{k}e_{0}$, temos que:

$||M^ke_{0}||_\infty \leq ||M^k||_{\infty}||e_{0}||_{\infty} \rightarrow ||e_{k}||_\infty \leq ||M^k||_{\infty}||e_{0}||_{\infty}$

Para que nosso $e_{k}$ convirja, precisamos que $||M^k||_{\infty} \leq \infty \rightarrow ||M||_{\infty} \leq 1$

Logo, como $||M||_\infty = \max\limits_{1 \leq i \leq n} \sum_{j=1}^{n}{| m_{i,j} |}$ (maior soma de valores absolutos das linhas), e nossa matriz $M = D^{-1}(L+U)$, temos que:

$||M||_\infty = \max\limits_{1 \leq i \leq n} \sum_{j=1}^{n}{| -\frac{a_{i,j}}{a_{j,j}} |} \rightarrow \max\limits_{1 \leq i \leq n} \sum_{j=1}^{n}{| -\frac{a_{i,j}}{a_{j,j}} |} \leq 1$ 

O que ocorre para as matrizes diagonais dominantes, dado que o elemento da diagonal $a_{j,j}$ é maior ou igual à soma dos elementos da linha, $a_{i,j}, i\neq j$. Assim, mostramos que, se a matriz for diagonal dominante, então o método de Jacobi converge.

## 09-08

#### [Método da Secante](https://en.wikipedia.org/wiki/Secant_method)

#### Gradiente Conjugado